# PTCB project

Report :
http://unil.im/Ly33D

Normalization :
http://unil.im/kdTj5

FeatureCounts tests :
http://unil.im/2sAq6

Read counts step by step :
http://unil.im/Ic1Wc

##### Notes / todo :
- Est-il possible de normaliser les datas avant de les passer dans IGV (Ils veulent retourner sur le visuel des pltos IGV après l'analyse DE) ?
- Extraire la liste des gènes d'intérêt de leur étude (mail Sandrine et mail Ophélie).
- Sur les tableaux de résultats : Avec réplicats, sortir un 2nd tableau qui ne contient que les DE significatifs (padj). Cas sans réplicats, sortir un 2nd tableau qui ne contient que les DE "possiblement significatifs", cad ceux qui restent après divers filtres (mean expression value = X, quartile log2, etc...).
- Sortir également tableaux de résultats réduit à la liste de gènes d'intérêt ?
- Créer heatmap avec resultats Pefanis + CRIBL pour tout les R et tout les S (avec en X les divers mutants + WT et en Y les top gènes DE (+ highlight les gènes de leur liste d'intérêt).
- Faire l'approche transcriptome via Kallisto. Donner la table de compte à DESEq2 puis croiser avec les résultats issus des comptes featureCounts => résultats de DE + robustes. Peut également nous permettre de tenter de faire la quantif des transcrits.
- QC step after mapping => rseqc.
- Harmoniser pipeline Pefanis & CRIBL.
- Comparaison avec résultats de Nice sur 1 jeu de données / reproduction exactes des param de Nice ?


- Réplicats soon : 1 R "Eµ FL KO" (rep de Yves EU), 1 R MAR Ko (rep de Yves) & 2 R cEµ KO.


- Steph : Intersection entre cas sur log2 + margin results.

## 1) Data

### 1.1) Raw reads

#### 1.1.1) CRIBL data

Is stored @Sophia Antipolis : <br>
Eric : 3B4, 3pRR, DEL, IRIS, PAL, WT. <br>
Yves : 3pRR, EU, MAR, WT. <br>
Replicates : EU-R (~Yves), MAR-R (~Yves), 2 CEU-R (new).

In [ ]:
### CODE ###
mkdir -p ~/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice;
cd ~/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice;

In [ ]:
### CODE ###
axel -q http://www.genomique.info/data/public/000-export/151109_pinaud.tar;
axel -q http://www.genomique.info/data/public/000-export/DEL_IRIS_fastq.tar;
axel -q http://www.genomique.info/data/public/000-export/160627_denizot.tar;
tar -xvf *.tar;

And some files renaming (for which I lost the script).

#### 1.1.2) Pefanis data

Retrieve all "BC" samples : https://www.ncbi.nlm.nih.gov/sra?term=SRP042355 :

In [ ]:
### CODE ###
for id in 'SRR1302208' 'SRR1302283' 'SRR1302414' \
           'SRR1302785' 'SRR1302786' 'SRR1302787' \
           'SRR1302788' 'SRR1302789' 'SRR1984164' \
           'SRR1984166' 'SRR1984167' 'SRR1984178';
do fastq-dump -O . --split-files $id;
done;

### 1.2) Mus musculus

#### 1.2.1) Reference genome & associated GFF

Downloaded from the "Genome Reference Consortium", latesst realease = GRCm38.p5 :

In [ ]:
### CODE ###
mkdir -p ~/Desktop/PTCB/BISCEm/Data/Mus_musculus;
cd ~/Desktop/PTCB/BISCEm/Data/Mus_musculus;

In [ ]:
### CODE ###
url='ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/635/GCA_000001635.7_GRCm38.p5';
axel -q $url/GCA_000001635.7_GRCm38.p5_genomic.fna.gz;
wget -nv $url/GCA_000001635.7_GRCm38.p5_genomic.gff.gz;
unpigz *_GRCm38.p5_genomic*;

Check if they contains the same sequences names :

In [ ]:
### CODE ###
grep '^>' GCA_000001635.7_GRCm38.p5_genomic.fna | sort -u > fna.txt
sed -i 's/>//' fna.txt 
awk -F "[[:space:]]" '{print $1}' fna.txt > fna_final.txt
awk -F "\t" '{if($0 !~ /^#/) print $1}' GCA_000001635.7_GRCm38.p5_genomic.gff | sort -u > gff.txt
diff fna_final.txt gff.txt

If the diff command produce no output, it's okay. <br>
One last thing, to avoid potential incompatilities in downstream analysis :

In [ ]:
### CODE ###
grep '^>' GCA_000001635.7_GRCm38.p5_genomic.fna | head -2;
awk -F "\t" '{if($0 !~ /^#/) print $1}' GCA_000001635.7_GRCm38.p5_genomic.gff | head -2;

What we have just shown is that sequences headers in the FNA file are more verbose than the associated name for the same sequence in the GFF file. Given that this extra info is not critical, let's discard it & check that we did this successfully :

In [ ]:
### CODE ###
gawk -i inplace -F "[[:space:]]" '{if($0 ~ /^>/) $0 = $1; print}' GCA_000001635.7_GRCm38.p5_genomic.fna;
grep '^>' GCA_000001635.7_GRCm38.p5_genomic.fna | head -2;
awk -F "\t" '{if($0 !~ /^#/) print $1}' GCA_000001635.7_GRCm38.p5_genomic.gff | head -2;

Load stuff on Genotoul :

In [ ]:
### CODE ###
sudo scp *_GRCm38.p5_genomic* escaon@genotoul.toulouse.inra.fr:/home/escaon/work/PTCB/Mus_musculus;

Remove useless files & compress the remaining for local storage :

In [ ]:
### CODE ###
rm *.txt;
pigz *_GRCm38.p5_genomic*;

#### 1.2.2) GTF annotations

Downloaded from "GENCODE" (for GRCm38.p5 genome). <br>
"It contains the comprehensive gene annotation on the reference chromosomes, scaffolds, assembly patches and alternate loci (haplotypes)". <br>
More precisely, it annotate a lot of gene/transcript biotypes : https://www.gencodegenes.org/gencode_biotypes.html.

In [ ]:
### CODE ###
axel -q ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/\
release_M14/gencode.vM14.chr_patch_hapl_scaff.annotation.gtf.gz;

## 2) Raw reads QC & filtering

### 2.1) FastQC

Setup working directory & paths :

In [ ]:
### CODE ###
reads_dir='/home/erwann/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice';
cd $reads_dir;
mkdir -p $reads_dir/QC;
mkdir -p ~/Software/FastQC/Outputs/PTCB;

Launch FastQC :

In [ ]:
### CODE ###
for f in *.fastq.gz;
do echo $f; fastqc --outdir ~/Software/FastQC/Outputs/PTCB -f fastq $f;
done >> $reads_dir/QC/fastqc.verbose;

### 2.2) MultiQC

We compile FastqQC analysis for all the data & at the sequencing-run level (EP & YD), to highlight potential run effect.

In [ ]:
### CODE ###
fastqc_dir='/home/erwann/Software/FastQC/Outputs/PTCB';
mkdir -p $reads_dir/QC;
multiqc $fastqc_dir \
        -n ptcb \
        -o $reads_dir/QC \
        -m fastqc -f > $reads_dir/QC/multiQC_ptcb.verbose 2>&1;
multiqc $fastqc_dir/*_EP_fastqc* \
        -n ptcb_ep \
        -o $reads_dir/QC \
        -m fastqc -f > $reads_dir/QC/multiQC_ptcb_ep.verbose 2>&1;
multiqc $fastqc_dir/*_YD_fastqc* \
        -n ptcb_yd \
        -o $reads_dir/QC \
        -m fastqc -f > $reads_dir/QC/multiQC_ptcb_yd.verbose 2>&1;

Visualize QC results :

In [ ]:
### CODE ###
chromium-browser $reads_dir/QC/*.html;

Conclusions :
Nice data overall. We can be a bit stringent during filtering to keep HQ reads.

### 2.3) Filtering

Trimmomatic 0.36

For both EP & YD files, FastQC detected Illumina 1.9 encoding. Referring to https://en.wikipedia.org/wiki/FASTQ_format, Illumina 1.8+ => Phred+33.

Sequences are trimmed with $TRIM = ~/Software/Trimmomatic_0.36/trimmomatic-0.36.jar.
Given MultiQC output, we trim starting and ending bases if their quality is < 25. We then trim with a sliding window of 5 bases, cutting the read at those bases if the mean quality drops below 25. Finally, we discard sequence < 50 nts.

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice;
mkdir -p ../Reads_raw_Nice_trim;
for id in '3B4_R_1_EP' '3B4_S_1_EP' '3pRR_R_1_EP' '3pRR_S_1_EP' \
          'DEL_R_1_EP' 'DEL_S_1_EP' 'IRIS_R_1_EP' 'IRIS_S_1_EP' \
          'PAL_R_1_EP' 'PAL_S_1_EP' 'WT_R_1_EP' 'WT_S_1_EP';
do java -jar $TRIM PE -threads 8 \
                      -phred33 \
                      $id.fastq.gz \
                      ${id%_*_*}'_2_EP.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_unpaired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_EP_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_EP_unpaired.fastq.gz' \
                      LEADING:25 TRAILING:25 SLIDINGWINDOW:5:25 MINLEN:50;
done &>> trim_EP.verbose;
for id in '3pRR_R_1_YD' '3pRR_S_1_YD' 'EU_R_1_YD' 'EU_S_1_YD' \
          'MAR_R_1_YD' 'MAR_S_1_YD' 'WT_R_1_YD' 'WT_S_1_YD';
do java -jar $TRIM PE -threads 8 \
                      -phred33 \
                      $id.fastq.gz \
                      ${id%_*_*}'_2_YD.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_unpaired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_YD_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_YD_unpaired.fastq.gz' \
                      LEADING:25 TRAILING:25 SLIDINGWINDOW:5:25 MINLEN:50;
done &>> trim_YD.verbose;

## 3) Mapping

On Genotoul via STAR, version 2.5.2b :

### 3.1) Build STAR index :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=12G
#$ -N star_idx
#$ -o /home/escaon/work/PTCB/star_idx.o
#$ -e /home/escaon/work/PTCB/star_idx.e

work='/home/escaon/work/PTCB/Mus_musculus';
star='/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR'
nt=8;

$star --runThreadN $nt \
      --runMode genomeGenerate \
      --genomeDir $work \
      --genomeFastaFiles $work/GCA_000001635.7_GRCm38.p5_genomic.fna \
      --sjdbGTFfile $work/GCA_000001635.7_GRCm38.p5_genomic.gff \
      --sjdbGTFtagExonParentTranscript Parent \
      --sjdbOverhang 150;

In [ ]:
### CODE ###
qsub -pe parallel_smp 8 star_idx.sh;

### 3.2) Map reads with STAR :

First script : star_3B4_R_EP.sh

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=10G
#$ -N star_3B4_R_EP
#$ -o /home/escaon/work/PTCB/star_3B4_R_EP.o
#$ -e /home/escaon/work/PTCB/star_3B4_R_EP.e

star='/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR';
save='/home/escaon/save/PTCB/Reads_raw_Nice_trim';
work='/home/escaon/work/PTCB';
nt=8;

for id in '3B4_R_EP';
do $star --runThreadN $nt \
         --genomeDir $work/Mus_musculus \
         --readFilesIn $save/${id%_*}'_1_EP_paired.fastq.gz' $save/${id%_*}'_2_EP_paired.fastq.gz' \
         --readFilesCommand zcat \
         --outFileNamePrefix $work/Mapped_vs_GRCm38.p5/$id'_' \
         --outSAMtype BAM SortedByCoordinate;
done;

In [ ]:
### CODE ###
qsub -pe parallel_smp 8 star_3B4_R_EP.sh;

Duplicate this script for others EP datasets & launch jobs :

In [ ]:
### CODE ###
for id in '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' 'DEL_R_EP' \
          'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' 'PAL_R_EP' \
          'PAL_S_EP' 'WT_R_EP' 'WT_S_EP';
do  cp star_3B4_R_EP.sh star_$id.sh;
    sed -i "s/3B4_R_EP/$id/" star_$id.sh;
    qsub -pe parallel_smp 8 star_$id.sh;
done;

Create a base script for YD, duplicate it & launch jobs :

In [ ]:
### CODE ###
cp star_WT_R_EP.sh star_WT_R_YD.sh;
grep --color 'EP' star_WT_R_YD.sh; # check if we are not going to destroy anything
sed -i "s/EP/YD/" star_WT_R_YD.sh;
qsub -pe parallel_smp 8 star_WT_R_YD.sh;

for id in '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' \
          'MAR_R_YD' 'MAR_S_YD' 'WT_S_YD';
do  cp star_WT_R_YD.sh star_$id.sh;
    sed -i "s/WT_R_YD/$id/" star_$id.sh;
    qsub -pe parallel_smp 8 star_$id.sh;
done;

### 3.3) Index BAM

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=40G
#$ -l h_vmem=48G
#$ -N samtools_idx
#$ -o /home/escaon/work/PTCB/samtools_idx.o
#$ -e /home/escaon/work/PTCB/samtools_idx.e

work='/home/escaon/work/PTCB/Mapped_vs_GRCm38.p5';

cd $work;
for f in *.bam;
do samtools index $f;
done;

In [ ]:
### CODE ###
qsub samtools_idx.sh;

### 3.3) RNA-seq alignment QC

http://rseqc.sourceforge.net

##### Retrieve BAMs locally :

In [ ]:
### CODE ###
mkdir -p ~/Desktop/PTCB/BISCEm/Outputs;
sudo scp -r escaon@genotoul.toulouse.inra.fr:/home/escaon/work/PTCB/Mapped_vs_GRCm38.p5 \
~/Desktop/PTCB/BISCEm/Outputs;

## 4) Counts

Before starting the counts, we need to create a "chr.aliases" file which link chrs names in GRCm38.p5 with chrs names in GENCODE annotations. This file was created from this assembly report :

In [ ]:
### CODE ###
wget -q ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/635/\
GCA_000001635.7_GRCm38.p5/GCA_000001635.7_GRCm38.p5_assembly_report.txt;

Count with featureCounts (Rsubread 1.24.2) :

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
cd $path/Outputs/Mapped_vs_GRCm38.p5;
for id in '3B4_R_EP' '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' \
          'DEL_R_EP' 'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' \
          'PAL_R_EP' 'PAL_S_EP' 'WT_R_EP' 'WT_S_EP' \
          '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' \
          'MAR_R_YD' 'MAR_S_YD' 'WT_R_YD' 'WT_S_YD';
do Rscript $path/Scripts/featureCounts.R $id;
done &>> $path/Outputs/FeatureCounts/featureCounts.verbose;

Content of featureCounts.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

args = commandArgs(trailingOnly=TRUE)
in_file = paste(args[1], 'bam', sep = '.')
out_file = paste('../FeatureCounts/', args[1], '.tsv', sep = '')

library(Rsubread)

fc = featureCounts(files=in_file,
                   annot.ext="../../Data/Mus_musculus/gencode.vM14.chr_patch_hapl_scaff.annotation.gtf",
                   isGTFAnnotationFile=TRUE,
                   GTF.featureType="exon",
                   GTF.attrType="gene_id",
                   chrAliases="../../Data/Mus_musculus/chr.aliases",
                   allowMultiOverlap=FALSE,
                   ignoreDup=FALSE,
                   isPairedEnd=TRUE,
                   nthreads=8,
                   minMQS=0,
                   maxMOp=10,
                   checkFragLength=FALSE,
                   useMetaFeatures=TRUE,
                   strandSpecific=2,
                   requireBothEndsMapped=TRUE,
                   countChimericFragments=FALSE,
                   primaryOnly=TRUE)
write.table(fc$counts, out_file, append= T, sep='\t', col.names=NA)

### 4.1) Downstream analysis compatibility
Slightly modify featureCounts output to make it compatible with future steps of the analysis.
We want our gene names to be the same than those contained in ensembl when retrieved with biomart.

Remove spreadsheet hidden ' " ' for downstream compatility with our awk manipulation :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Outputs/FeatureCounts;
for f in *.tsv;
do sed 's/\"//g' $f > ${f%.*}".temp.tsv";
done;

Use awk sub to convert "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" :

In [ ]:
### CODE ###
for f in *.temp.tsv;
do awk -F '\t' -v OFS='\t' 'NR > 1 {sub(/\.[0-9]*/, "", $1)} 1' $f > ${f%%.*}"_ensembl_gene_id.tsv";
done;
rm *.temp.tsv;

Check that the removal of ".[0-9]?" from the gene id column did not change the number of unique gene ids :

In [ ]:
### CODE ###
for f in *.tsv;
do awk -F '\t' '{print $1}' $f | sort -u | wc -l;
done;

We did show that transforming gene name "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" did not change the number unique gene names.

## 5) DE analysis

Perfomed with DESeq2_1.16.1 (using Bioconductor 3.5 (BiocInstaller 1.26.0), R 3.4.1 (2017-06-30)) :

### 5.1) Resting "R" vs Stimulated "S" for the same mouse model (MAR, DEL ...)

##### With no replicates :

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
cd $path/Outputs/DESeq2;
Rscript $path/Scripts/deseq2_R_vs_S.R &>> $path/Outputs/DESeq2/deseq2_R_vs_S.verbose;

Content of deseq2_R_vs_S_no_replicates.R for the first comparison :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

# Open featureCounts results for the same mouse model & experiment
threeB4_R_EP = read.table("../FeatureCounts/3B4_R_EP_ensembl_gene_id.tsv")
threeB4_S_EP = read.table("../FeatureCounts/3B4_S_EP_ensembl_gene_id.tsv")
threeB4_EP = data.frame(row.names = row.names(threeB4_R_EP),
                        threeB4_R_EP = threeB4_R_EP$X3B4_R_EP.bam,
                        threeB4_S_EP = threeB4_S_EP$X3B4_S_EP.bam)
threeB4_EP$threeB4_R_EP=as.numeric(as.character(threeB4_EP$threeB4_R_EP))
threeB4_EP$threeB4_S_EP=as.numeric(as.character(threeB4_EP$threeB4_S_EP))
# Create design
condition <- c('R', 'S')
type <- c('paired-end', 'paired-end')
threeB4_EP_infos <- data.frame(condition, type)
row.names(threeB4_EP_infos) <- c('threeB4_R_EP', 'threeB4_S_EP')
# Sanity check
head(threeB4_EP)
tail(threeB4_EP)
threeB4_EP_infos
all(rownames(threeB4_EP_infos) %in% colnames(threeB4_EP))
# Start DESeq2 analysis
library("DESeq2")
dds <- DESeqDataSetFromMatrix(countData = threeB4_EP,
                              colData = threeB4_EP_infos,
                              design = ~ condition)
# Sanity check
dds
# Pre-filtering removing rows that have only 0 or 1 read
dds <- dds[ rowSums(counts(dds)) > 1, ]
# Check on many genes it discarded
summary(dds)
# Set factor to tell which one is the control group
dds$condition <- factor(dds$condition, levels=c('R','S'))
# Start actual DE analysis
dds <- DESeq(dds)
res <- results(dds)
# We are using DESeq2_1.16.1, thus betaPrior is set to 'FALSE' 
# and we need to obtain shrunken LFCs with function ‘lfcShrink’.
# https://support.bioconductor.org/p/95695/
res <- lfcShrink(dds,
       contrast=c("condition","R","S"),
       res=res)
summary(res, alpha=0.05)
# MA plot : the log2 of ratio of the counts between two conditions is represented 
# as a function of the log2 of the mean of the counts
png('EP_3B4_R_vs_S_MA.png')
plotMA(res,
       ylim=c(min(res$log2FoldChange),
        max(res$log2FoldChange)))
dev.off()
# Volcano plot : the log10 of the adjusted pvalue is represented
# as a function of the log2 of ratio of the counts.
png('EP_3B4_R_vs_S_volcano.png')
plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")
sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()
# Retrieve normalized counts for each condition for future merge
dds <- estimateSizeFactors(dds)
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]
colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "R_normalized_count"
colnames(nor_counts_df)[3] <- "S_normalized_count"
# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"
# Check
summary(res)
head(res)
# Add biotypes infos
# Get ensembl infos
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                        dataset = "mmusculus_gene_ensembl",
                        host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype"),
                                            mart = mart)
# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)
# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_gene_biotype"
# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)
# Save the result in a TSV
write.table(res_biotyped_counts,
            '../DESeq2/EP_3B4_R_vs_S.tsv',
            append=F, sep='\t',
            col.names=T,
            row.names=F)

##### With replicates when available (i.e., 1 sample coming from Eric Pinaud "EP" and 1 from Yves Denizot "YD") :

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
cd $path/Outputs/DESeq2;
Rscript $path/Scripts/deseq2_pooled_R_vs_S.R &>> $path/Outputs/DESeq2/deseq2_pooled_R_vs_S.verbose;

Content of deseq2_R_vs_S_with_replicates.R for the first comparison :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

# Open featureCounts results for the same mouse model regardless of the experiment
WT_R_EP = read.table("../FeatureCounts/WT_R_EP_ensembl_gene_id.tsv")
WT_R_YD = read.table("../FeatureCounts/WT_R_YD_ensembl_gene_id.tsv")
WT_S_EP = read.table("../FeatureCounts/WT_S_EP_ensembl_gene_id.tsv")
WT_S_YD = read.table("../FeatureCounts/WT_S_YD_ensembl_gene_id.tsv")
WT_pooled = data.frame(row.names = row.names(WT_R_EP),
                       WT_R_EP = WT_R_EP$WT_R_EP.bam,
                       WT_R_YD = WT_R_YD$WT_R_YD.bam,
                       WT_S_EP = WT_S_EP$WT_S_EP.bam,
                       WT_S_YD = WT_S_YD$WT_S_YD.bam)
WT_pooled$WT_R_EP=as.numeric(as.character(WT_pooled$WT_R_EP))
WT_pooled$WT_R_YD=as.numeric(as.character(WT_pooled$WT_R_YD))
WT_pooled$WT_S_EP=as.numeric(as.character(WT_pooled$WT_S_EP))
WT_pooled$WT_S_YD=as.numeric(as.character(WT_pooled$WT_S_YD))
# Create design
condition <- c('R', 'R', 'S', 'S')
type <- c('paired-end', 'paired-end', 'paired-end', 'paired-end')
WT_pooled_infos <- data.frame(condition, type)
row.names(WT_pooled_infos) <- c('WT_R_EP', 'WT_R_YD', 'WT_S_EP', 'WT_S_YD')
# Sanity check
head(WT_pooled)
tail(WT_pooled)
WT_pooled_infos
all(rownames(WT_pooled_infos) %in% colnames(WT_pooled))
# Start DESeq2 analysis
library("DESeq2")
dds <- DESeqDataSetFromMatrix(countData = WT_pooled,
                              colData = WT_pooled_infos,
                              design = ~ condition)
# Sanity check
dds
# Pre-filtering removing rows that have only 0 or 1 read
dds <- dds[ rowSums(counts(dds)) > 1, ]
# Check on many genes it discarded
summary(dds)
# Set factor to tell which one is the control group
dds$condition <- factor(dds$condition, levels=c('R','S'))
# Start actual DE analysis
dds <- DESeq(dds)
res <- results(dds)
# We are using DESeq2_1.16.1, thus betaPrior is set to 'FALSE' 
# and we need to obtain shrunken LFCs with function ‘lfcShrink’.
# https://support.bioconductor.org/p/95695/
res <- lfcShrink(dds,
       contrast=c("condition","R","S"),
       res=res)
summary(res, alpha=0.05)
# MA plot : the log2 of ratio of the counts between two conditions is represented
# as a function of the log2 of the mean of the counts
png('Pooled_WT_R_vs_S_MA.png')
plotMA(res,
       ylim=c(min(res$log2FoldChange),
        max(res$log2FoldChange)))
dev.off()
# Volcano plot : the log10 of the adjusted pvalue is represented
# as a function of the log2 of ratio of the counts.
png('Pooled_WT_R_vs_S_volcano.png')
plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")
sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()
# Retrieve normalized counts for each condition for future merge
dds <- estimateSizeFactors(dds)
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]
colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "R_EP_normalized_count"
colnames(nor_counts_df)[3] <- "R_YD_normalized_count"
colnames(nor_counts_df)[4] <- "S_EP_normalized_count"
colnames(nor_counts_df)[5] <- "S_YD_normalized_count"
# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"
# Check
summary(res)
head(res)
# Add biotypes infos
# Get ensembl infos
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                        dataset = "mmusculus_gene_ensembl",
                        host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype"),
                                            mart = mart)
# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)
# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_gene_biotype"
# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)
# Save the result in a TSV
write.table(res_biotyped_counts,
            '../DESeq2/Pooled_WT_R_vs_S.tsv',
            append=F, sep='\t',
            col.names=T,
            row.names=F)

##### Pooled R vs S with all mouse models & WT (EP, YD, EP+YD) :

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
cd $path/Outputs/DESeq2;
Rscript $path/Scripts/deseq2_pooled_all_R_vs_all_S.R \
&>> $path/Outputs/DESeq2/deseq2_pooled_all_R_vs_all_S.verbose;

Content of deseq2_R_vs_S_pooled.R for the first comparison :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

############################
### all_EP_R vs all_EP_S ###
############################
# Open featureCounts results for the same mouse model regardless of the experiment
threeB4_R_EP = read.table("../FeatureCounts/3B4_R_EP_ensembl_gene_id.tsv")
threepRR_R_EP = read.table("../FeatureCounts/3pRR_R_EP_ensembl_gene_id.tsv")
DEL_R_EP = read.table("../FeatureCounts/DEL_R_EP_ensembl_gene_id.tsv")
IRIS_R_EP = read.table("../FeatureCounts/IRIS_R_EP_ensembl_gene_id.tsv")
PAL_R_EP = read.table("../FeatureCounts/PAL_R_EP_ensembl_gene_id.tsv")
WT_R_EP = read.table("../FeatureCounts/WT_R_EP_ensembl_gene_id.tsv")
threeB4_S_EP = read.table("../FeatureCounts/3B4_S_EP_ensembl_gene_id.tsv")
threepRR_S_EP = read.table("../FeatureCounts/3pRR_S_EP_ensembl_gene_id.tsv")
DEL_S_EP = read.table("../FeatureCounts/DEL_S_EP_ensembl_gene_id.tsv")
IRIS_S_EP = read.table("../FeatureCounts/IRIS_S_EP_ensembl_gene_id.tsv")
PAL_S_EP = read.table("../FeatureCounts/PAL_S_EP_ensembl_gene_id.tsv")
WT_S_EP = read.table("../FeatureCounts/WT_S_EP_ensembl_gene_id.tsv")
all_pooled = data.frame(row.names = row.names(threeB4_R_EP),
                        threeB4_R_EP = threeB4_R_EP$X3B4_R_EP.bam,
                        threepRR_R_EP = threepRR_R_EP$X3pRR_R_EP.bam,
                        DEL_R_EP = DEL_R_EP$DEL_R_EP.bam,
                        IRIS_R_EP = IRIS_R_EP$IRIS_R_EP.bam,
                        PAL_R_EP = PAL_R_EP$PAL_R_EP.bam,
                        WT_R_EP =  WT_R_EP$WT_R_EP.bam,
                        threeB4_S_EP = threeB4_S_EP$X3B4_S_EP.bam,
                        threepRR_S_EP = threepRR_S_EP$X3pRR_S_EP.bam,
                        DEL_S_EP = DEL_S_EP$DEL_S_EP.bam,
                        IRIS_S_EP = IRIS_S_EP$IRIS_S_EP.bam,
                        PAL_S_EP = PAL_S_EP$PAL_S_EP.bam,
                        WT_S_EP = WT_S_EP$WT_S_EP.bam)
all_pooled$threeB4_R_EP=as.numeric(as.character(all_pooled$threeB4_R_EP))
all_pooled$threepRR_R_EP=as.numeric(as.character(all_pooled$threepRR_R_EP))
all_pooled$DEL_R_EP=as.numeric(as.character(all_pooled$DEL_R_EP))
all_pooled$IRIS_R_EP=as.numeric(as.character(all_pooled$IRIS_R_EP))
all_pooled$PAL_R_EP=as.numeric(as.character(all_pooled$PAL_R_EP))
all_pooled$WT_R_EP=as.numeric(as.character(all_pooled$WT_R_EP))
all_pooled$threeB4_S_EP=as.numeric(as.character(all_pooled$threeB4_S_EP))
all_pooled$threepRR_S_EP=as.numeric(as.character(all_pooled$threepRR_S_EP))
all_pooled$DEL_S_EP=as.numeric(as.character(all_pooled$DEL_S_EP))
all_pooled$IRIS_S_EP=as.numeric(as.character(all_pooled$IRIS_S_EP))
all_pooled$PAL_S_EP=as.numeric(as.character(all_pooled$PAL_S_EP))
all_pooled$WT_S_EP=as.numeric(as.character(all_pooled$WT_S_EP))
# Create design
condition <- c('R', 'R', 'R', 'R', 'R', 'R', 'S', 'S', 'S', 'S', 'S', 'S')
type <- c('paired-end', 'paired-end', 'paired-end', 'paired-end', 'paired-end', 'paired-end',
          'paired-end', 'paired-end', 'paired-end', 'paired-end', 'paired-end', 'paired-end')
all_pooled_infos <- data.frame(condition, type)
row.names(all_pooled_infos) <- c('threeB4_R_EP', 'threepRR_R_EP', 'DEL_R_EP', 'IRIS_R_EP', 
                                 'PAL_R_EP', 'WT_R_EP', 'threeB4_S_EP', 'threepRR_S_EP', 
                                 'DEL_S_EP', 'IRIS_S_EP', 'PAL_S_EP', 'WT_S_EP')
# Sanity check
head(all_pooled)
tail(all_pooled)
all_pooled_infos
all(rownames(all_pooled_infos) %in% colnames(all_pooled))
# Start DESeq2 analysis
library("DESeq2")
dds <- DESeqDataSetFromMatrix(countData = all_pooled,
                              colData = all_pooled_infos,
                              design = ~ condition)
# Sanity check
dds
# Pre-filtering removing rows that have only 0 or 1 read
dds <- dds[ rowSums(counts(dds)) > 1, ]
# Check on many genes it discarded
summary(dds)
# Set factor to tell which one is the control group
dds$condition <- factor(dds$condition, levels=c('R','S'))
# Start actual DE analysis
dds <- DESeq(dds)
res <- results(dds)
# We are using DESeq2_1.16.1, thus betaPrior is set to 'FALSE' 
# and we need to obtain shrunken LFCs with function ‘lfcShrink’.
# https://support.bioconductor.org/p/95695/
res <- lfcShrink(dds,
                 contrast=c("condition","R","S"),
                 res=res)
summary(res, alpha=0.05)
# MA plot : the log2 of ratio of the counts between two conditions is represented 
# as a function of the log2 of the mean of the counts
png('pooled_all_R_EP_vs_all_S_EP_MA.png')
plotMA(res,
       ylim=c(min(res$log2FoldChange),
        max(res$log2FoldChange)))
dev.off()
# Volcano plot : the log10 of the adjusted pvalue is represented as a function 
# of the log2 of ratio of the counts.
png('pooled_all_R_EP_vs_all_S_EP_volcano.png')
plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")
sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()
# Retrieve normalized counts for each condition for future merge
dds <- estimateSizeFactors(dds)
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]
colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "threeB4_R_normalized_count"
colnames(nor_counts_df)[3] <- "threepRR_R_normalized_count"
colnames(nor_counts_df)[4] <- "DEL_R_normalized_count"
colnames(nor_counts_df)[5] <- "IRIS_R_normalized_count"
colnames(nor_counts_df)[6] <- "PAL_R_normalized_count"
colnames(nor_counts_df)[7] <- "WT_R_normalized_count"
colnames(nor_counts_df)[8] <- "threeB4_S_normalized_count"
colnames(nor_counts_df)[9] <- "threepRR_S_normalized_count"
colnames(nor_counts_df)[10] <- "DEL_S_normalized_count"
colnames(nor_counts_df)[11] <- "IRIS_S_normalized_count"
colnames(nor_counts_df)[12] <- "PAL_S_normalized_count"
colnames(nor_counts_df)[13] <- "WT_S_normalized_count"
# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"
# Check
summary(res)
head(res)
# Add biotypes infos
# Get ensembl infos
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                         dataset = "mmusculus_gene_ensembl",
                         host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype"),
                                            mart = mart)
# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)
# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_gene_biotype"
# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)
# Save the result in a TSV
write.table(res_biotyped_counts,
            '../DESeq2/pooled_all_R_EP_vs_all_S_EP.tsv',
            append=F, sep='\t',
            col.names=T,
            row.names=F)

### 5.2) WT vs Mutant (for R and for S)

##### With no replicates :

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
cd $path/Outputs/DESeq2;
Rscript $path/Scripts/deseq2_mutant_vs_wt.R &>> $path/Outputs/DESeq2/deseq2_mutant_vs_wt2.verbose;

Content of deseq2_mutant_vs_wt_no_replicates.R for the first comparison :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

# Open featureCounts results for the same mouse model & experiment
WT_R_EP = read.table("../FeatureCounts/WT_R_EP_ensembl_gene_id.tsv")
threeB4_R_EP = read.table("../FeatureCounts/3B4_R_EP_ensembl_gene_id.tsv")
both_EP = data.frame(row.names = row.names(WT_R_EP),
                        WT_R_EP = WT_R_EP$WT_R_EP.bam,
                        threeB4_R_EP = threeB4_R_EP$X3B4_R_EP.bam)
both_EP$WT_R_EP=as.numeric(as.character(both_EP$WT_R_EP))
both_EP$threeB4_R_EP=as.numeric(as.character(both_EP$threeB4_R_EP))
# Create design
condition <- c('WT', 'MUTANT')
type <- c('paired-end', 'paired-end')
both_EP_infos <- data.frame(condition, type)
row.names(both_EP_infos) <- c('WT_R_EP', 'threeB4_R_EP')
# Sanity check
head(both_EP)
tail(both_EP)
both_EP_infos
all(rownames(both_EP_infos) %in% colnames(both_EP))
# Start DESeq2 analysis
library("DESeq2")
dds <- DESeqDataSetFromMatrix(countData = both_EP,
                              colData = both_EP_infos,
                              design = ~ condition)
# Sanity check
dds
# Pre-filtering removing rows that have only 0 or 1 read
dds <- dds[ rowSums(counts(dds)) > 1, ]
# Check on many genes it discarded
summary(dds)
# Set factor to tell which one is the control group
dds$condition <- factor(dds$condition, levels=c('WT','MUTANT'))
# Start actual DE analysis
dds <- DESeq(dds)
res <- results(dds)
# We are using DESeq2_1.16.1, thus betaPrior is set to 'FALSE'
# and we need to obtain shrunken LFCs with function ‘lfcShrink’.
# https://support.bioconductor.org/p/95695/
res <- lfcShrink(dds,
       contrast=c("condition","WT","MUTANT"),
       res=res)
summary(res, alpha=0.05)
# MA plot : the log2 of ratio of the counts between two conditions is represented
# as a function of the log2 of the mean of the counts
png('EP_R_3B4_vs_WT_MA.png')
plotMA(res,
       ylim=c(min(res$log2FoldChange),
        max(res$log2FoldChange)))
dev.off()
# Volcano plot : the log10 of the adjusted pvalue is represented
# as a function of the log2 of ratio of the counts.
png('EP_R_3B4_vs_WT_volcano.png')
plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")
sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()
# Retrieve normalized counts for each condition for future merge
dds <- estimateSizeFactors(dds)
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]
colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "WT_normalized_count"
colnames(nor_counts_df)[3] <- "threeB4_normalized_count"
# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"
# Check
summary(res)
head(res)
# Add biotypes infos
# Get ensembl infos
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                        dataset = "mmusculus_gene_ensembl",
                        host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype"),
                                            mart = mart)
# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)
# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_gene_biotype"
# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)
# Save the result in a TSV
write.table(res_biotyped_counts,
            '../DESeq2/EP_R_3B4_vs_WT.tsv',
            append=F, sep='\t',
            col.names=T,
            row.names=F)

##### With replicates when available (3pRR vs WT) :

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
cd $path/Outputs/DESeq2;
Rscript $path/Scripts/deseq2_pooled_mutant_vs_wt.R &>> $path/Outputs/DESeq2/deseq2_pooled_mutant_vs_wt.verbose;

Content of deseq2_mutant_vs_wt_with_replicates.R for the first comparison :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

# Open featureCounts results for the same mouse model regardless of the experiment
WT_R_EP = read.table("../FeatureCounts/WT_R_EP_ensembl_gene_id.tsv")
WT_R_YD = read.table("../FeatureCounts/WT_R_YD_ensembl_gene_id.tsv")
threepRR_R_EP = read.table("../FeatureCounts/3pRR_R_EP_ensembl_gene_id.tsv")
threepRR_R_YD = read.table("../FeatureCounts/3pRR_R_YD_ensembl_gene_id.tsv")
all_pooled = data.frame(row.names = row.names(WT_R_EP),
             WT_R_EP = WT_R_EP$WT_R_EP.bam,
             WT_R_YD = WT_R_YD$WT_R_YD.bam,
             threepRR_R_EP = threepRR_R_EP$X3pRR_R_EP.bam,
             threepRR_R_YD = threepRR_R_YD$X3pRR_R_YD.bam)
all_pooled$WT_R_EP=as.numeric(as.character(all_pooled$WT_R_EP))
all_pooled$WT_R_YD=as.numeric(as.character(all_pooled$WT_R_YD))
all_pooled$threepRR_R_EP=as.numeric(as.character(all_pooled$threepRR_R_EP))
all_pooled$threepRR_R_YD=as.numeric(as.character(all_pooled$threepRR_R_YD))
# Create design
condition <- c('WT', 'WT', 'MUTANT', 'MUTANT')
type <- c('paired-end', 'paired-end', 'paired-end', 'paired-end')
all_pooled_infos <- data.frame(condition, type)
row.names(all_pooled_infos) <- c('WT_R_EP', 'WT_R_YD', 'threepRR_R_EP', 'threepRR_R_YD')
# Sanity check
head(all_pooled)
tail(all_pooled)
all_pooled_infos
all(rownames(all_pooled_infos) %in% colnames(all_pooled))
# Start DESeq2 analysis
library("DESeq2")
dds <- DESeqDataSetFromMatrix(countData = all_pooled,
                colData = all_pooled_infos,
                design = ~ condition)
# Sanity check
dds
# Pre-filtering removing rows that have only 0 or 1 read
dds <- dds[ rowSums(counts(dds)) > 1, ]
# Check on many genes it discarded
summary(dds)
# Set factor to tell which one is the control group
dds$condition <- factor(dds$condition, levels=c('WT','MUTANT'))
# Start actual DE analysis
dds <- DESeq(dds)
res <- results(dds)
# We are using DESeq2_1.16.1, thus betaPrior is set to 'FALSE'
# and we need to obtain shrunken LFCs with function ‘lfcShrink’.
# https://support.bioconductor.org/p/95695/
res <- lfcShrink(dds,
       contrast=c("condition","WT","MUTANT"),
       res=res)
summary(res, alpha=0.05)
# MA plot : the log2 of ratio of the counts between two conditions is represented
# as a function of the log2 of the mean of the counts
png('Pooled_R_3pRR_vs_WT_MA.png')
plotMA(res,
       ylim=c(min(res$log2FoldChange),
        max(res$log2FoldChange)))
dev.off()
# Volcano plot : the log10 of the adjusted pvalue is represented
# as a function of the log2 of ratio of the counts.
png('Pooled_R_3pRR_vs_WT.png')
plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")
sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()
# Retrieve normalized counts for each condition for future merge
dds <- estimateSizeFactors(dds)
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]
colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "WT_EP_normalized_count"
colnames(nor_counts_df)[3] <- "WT_YD_normalized_count"
colnames(nor_counts_df)[4] <- "threepRR_EP_normalized_count"
colnames(nor_counts_df)[5] <- "threepRR_YD_normalized_count"
# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"
# Check
summary(res)
head(res)
# Add biotypes infos
# Get ensembl infos
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                        dataset = "mmusculus_gene_ensembl",
                        host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype"),
                                            mart = mart)
# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)
# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_gene_biotype"
# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)
# Save the result in a TSV
write.table(res_biotyped_counts,
            '../DESeq2/Pooled_R_3pRR_vs_WT.tsv',
            append=F, sep='\t',
            col.names=T,
            row.names=F)

### 5.3) Mutant vs Mutant

##### With no replicates :

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
cd $path/Outputs/DESeq2;
Rscript $path/Scripts/script.R &>> $path/Outputs/DESeq2/script.verbose;

Content of deseq2_mutant_vs_mutant_no_replicates.R for the first comparison :

In [ ]:
INCOMING

##### With replicates when available :

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
cd $path/Outputs/DESeq2;
Rscript $path/Scripts/script.R &>> $path/Outputs/DESeq2/script.verbose;

Content of deseq2_mutant_vs_mutant_with_replicates.R for the first comparison :

In [ ]:
INCOMING

### 5.4) Heatmap

In [15]:
path='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $path;
Rscript $path/Scripts/deseq2_heatmap.R &>> $path/Outputs/DESeq2/deseq2_heatmap.verbose;

### 5.5) Output formatting

##### Point to Comma :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2;
mkdir -p Formatted_TSV/Comma;

In [ ]:
### CODE ###
for f in *.tsv;
do sed 's/\./,/g' $f > Formatted_TSV/Comma${f%.*}'_comma.tsv';
done;

##### Top DE genes (no-replicates) :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2;
mkdir -p Formatted_TSV/Filtered_top_DE;

In [ ]:
### CODE ###


##### Top DE genes (with-replicates) :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2;
mkdir -p Formatted_TSV/Filtered_top_DE;